In [ ]:
import numpy as np
import seaborn as sb
import pandas
import sys
import itertools
import matplotlib.pyplot as plt
import nltk
import csv
import datetime
import collections
import time
%matplotlib notebook

# TFLearn

For those, for whom tensorflow is a bit daring, I highly recommend the abstraction layer tflearn built on top of it.

http://tflearn.org/

## TFLearn - CNN on MNIST
The following is a fully self-contained example that downloads MNIST (man, I have so many versions of this on my computer now!) and trains a standard CONV-net on it.

In [ ]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

# Data loading and preprocessing
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True)
X = X.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])

# Building convolutional network
network = input_data(shape=[None, 28, 28, 1], name='input')
network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
# even fancier normalization of the response - this helps stabilize the
# responses for large nets
network = local_response_normalization(network)
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = fully_connected(network, 1024, activation='tanh')
network = dropout(network, 0.5)
network = fully_connected(network, 10, activation='softmax')
network = regression(network, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target')

# Training
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit({'input': X}, {'target': Y}, n_epoch=2,
           validation_set=({'input': testX}, {'target': testY}),
           snapshot_step=500, show_metric=True, run_id='convnet_mnist')

## TFLearn - LSTM on Shakespeare

In [ ]:
import os
import pickle

import tflearn
from tflearn.data_utils import *

path = "data/shakespeare_plays.txt"
char_idx_file = 'char_idx.pickle'

maxlen = 25

char_idx = None
if os.path.isfile(char_idx_file):
    print('Loading previous char_idx')
    char_idx = pickle.load(open(char_idx_file, 'rb'))

X, Y, char_idx = textfile_to_semi_redundant_sequences(path, seq_maxlen=maxlen, 
                                                      redun_step=20,pre_defined_char_idx=char_idx)

# this writes the character dictionary onto disk
# we can reload this next time to save the pre-processing
pickle.dump(char_idx, open(char_idx_file,'wb'))

# now let's define the input tensor
g = tflearn.input_data([None, maxlen, len(char_idx)])
# this is a three-layer LSTM with 512 hidden nodes each
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512)
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, len(char_idx), activation='softmax')
g = tflearn.regression(g, optimizer='adam', loss='categorical_crossentropy',
                       learning_rate=0.001)

m = tflearn.SequenceGenerator(g, dictionary=char_idx,
                              seq_maxlen=maxlen,
                              clip_gradients=5.0,
                              checkpoint_path='model_shakespeare')

for i in range(50):
    seed = random_sequence_from_textfile(path, maxlen)
    m.fit(X, Y, validation_set=0.1, batch_size=128, n_epoch=1, run_id='shakespeare')
    print("-- TESTING...")
    print("-- Test with temperature of 1.0 --")
    print(m.generate(600, temperature=1.0, seq_seed=seed))
    print("-- Test with temperature of 0.5 --")
    print(m.generate(600, temperature=0.5, seq_seed=seed))

Training Step: 321  | total loss: 3.18723 | time: 545.033s
| Adam | epoch: 001 | loss: 3.18723 -- iter: 041088/235149


In [ ]:
X.shape